In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os,sys, io, re, time
from glob import glob
from collections import OrderedDict
import itertools
from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.rail_env import RailEnv
from flatland.envs.persistence import RailEnvPersister
from flatland.utils.rendertools import RenderTool
import flatland.utils.jupyter_utils as ju
from PIL import Image
import networkx as nx
#import rustworkx as nx
import json
from numpy import array
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from matplotlib import cm
from IPython.display import display, HTML
from ipywidgets import Button, HBox, VBox
from flatland.graphs.graph_utils import RailEnvGraph, trans_int_to_4x4, trans_int_to_binstr, trans_int_to_nesw, get_rail_transitions_df
from flatland.graphs.graph_utils import get_simple_path, plotGraphEnv
import flatland.graphs.graph_utils as gu
# from flatland.envs.rail_generators import rail_from_manual_specifications_generator
#from flatland.envs.rail_generators import random_rail_generator, complex_rail_generator, sparse_rail_generator, empty_rail_generator
from flatland.envs.rail_generators import sparse_rail_generator, empty_rail_generator
from flatland.envs.observations import GlobalObsForRailEnv
from flatland.envs.rail_generators import sparse_rail_generator
#from flatland.envs.schedule_generators import sparse_schedule_generator
from flatland.envs.line_generators import sparse_line_generator
from flatland.envs.malfunction_generators import malfunction_from_params, MalfunctionParameters

In [ ]:
from flatland.utils import env_edit_utils as eeu
from flatland.utils import jupyter_utils as ju
import ipywidgets as ipw

In [ ]:
#import torch
#from torch import tensor
#from functools import partial
#from flatland_contrib.graphs  import graphpath as gp
#from ijsolution import graphpath as gp
#from ijsolution import PathUtils as pu

In [ ]:
#display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:

if False:
    dfMD = pd.read_csv("../../jeremyw/scratch/test-envs/metadata.csv")
    dfMD2 = dfMD[["test_id", "n_agents", "x_dim", "y_dim"]] # , "max_rails_in_city", "max_rails_between_cities" ]]
    dfMD2.drop_duplicates()[:5]
    #sDirEnvs = "../../neurips2020-flatland-scoring/environments/neurips2020_round_2_v6_final/debug/"
    #sDirEnvs = "../../neurips2020-flatland-starter-kit/scratch/test-envs/"
    sDirEnvs = "../../jeremyw/scratch/test-envs/"
    lsTests = glob(sDirEnvs + "Test_*")
    lsTests.sort(key=lambda s: int(os.path.basename(s)[5:]))  # list of tests eg Test_1, Test_20 sorted in numerical order, ie 1 < 9 < 10 < 20 etc (without expecting leading zeros)

    lsTestLev = []
    for sDirTest in lsTests:
        lsLevs = glob(sDirTest + "/Lev*")
        lsLevs.sort(key=lambda s: int(os.path.basename(s)[6:-4]))  # sort the levels in numerical order, again without leading zeros
        for sEnv in lsLevs:
            lsTestLev.append(sEnv)
    print(len(lsTestLev), lsTestLev[-1])
if False:
    llMD = []
    for sTestLev in lsTestLev[:50]:
        env, env_dict=RailEnvPersister.load_new(sTestLev)
        lRow = (re.search("/([^/]+)/([^/]+)$", sTestLev)[0], env.width, env.height, len(env.agents), env.malfunction_process_data.malfunction_rate) 
        #print(os.path.basename(sTestLev), env.width, env.height, len(env.agents), env.malfunction_process_data.malfunction_rate)
        llMD.append(lRow)
    pd.DataFrame(llMD, columns=["file", "width", "height", "agents", "mf_rate"])[18:25]
if False:
    #env, envModel = eeu.makeTestEnv("loop_with_loops", nAg=10, bUCF=True)
    #env, envModel = eeu.makeTestEnv(lsEnvNames[3], nAg=10, bUCF=True)
    #env, env_dict=RailEnvPersister.load_new("Test_20_Level_0.pkl")
    iTest = 12 # 30
    sTestLev = lsTestLev[iTest]
    print(sTestLev)
    env, env_dict=RailEnvPersister.load_new(sTestLev)

In [ ]:
env, oModel = eeu.makeTestEnv("alternative_needs_pause")

In [ ]:
#behaviour = ju.ForwardWithPause(env, [ju.AgentPause(0, 10, 5)])
behaviour = ju.AlwaysForward(env)
oEC = ju.EnvCanvas(env, behaviour, ) #size=(1000,1000))
env.reset(regenerate_rail=False)
oEC.show()
lImages = []

for i in range(25):
    oEC.step()
    oEC.render()
    time.sleep(0.1)
    aImg = oEC.oRT.get_image()
    lImages.append(aImg)

In [ ]:
aImg = oEC.oRT.get_image()

In [ ]:
type(aImg), aImg.shape

In [ ]:
oReg = gu.RailEnvGraph(env)
G2 = oReg.graph_rail_grid()

In [ ]:
if True:
    #plotGraphEnv(G2, env, aImg)
    plotGraphEnv(oReg.G, env, aImg, figsize=(10,10), node_size=10,
                    node_colors={"rail":"blue", "grid":"red"},
                    edge_colors={"hold":"gray", "dir":"green"},
                    show_nodes=("rail", "grid"),
                    show_edges=("dir", "hold")
                   )
    with io.BytesIO() as fbio:
        plt.savefig(fbio, bbox_inches="tight", pad_inches=0)
        bimgEnv = fbio.getvalue()
    plt.close()

In [ ]:
plotGraphEnv(oReg.G, env, aImg, figsize=(10,10), node_size=10,
                    node_colors={"rail":"blue", "grid":"red"},
                    edge_colors={"hold":"gray", "dir":"green"},
                    show_nodes=("rail", "grid"),
                    show_edges=("dir", "hold")
                   )

### Reduced Paths G5
Comment out whether you want G5 "reduced" or the full G2

In [ ]:
if False:
    G5 = oReg.reduce_simple_paths()
else:
    G5 = G2


In [ ]:
#plotGraphEnv(G2, env, aImg)
plotGraphEnv(G5, env, aImg, figsize=(10,10), node_size=10,
                space=0.3,
                node_colors={"rail":"blue", "grid":"red"},
                edge_colors={"hold":"gray", "dir":"green"},
                show_nodes=("rail",
                            # "grid"
                            ),
                show_edges=("dir", 
                            #"hold"
                            ),
                #show_edge_weights=True,
                show_labels=(),
                alpha_img=0.8
               )
with io.BytesIO() as fbOut:
    plt.savefig(fbOut, bbox_inches="tight", pad_inches=0)
    bimgReduced = fbOut.getvalue()
#plt.close()

Here it is with the simple paths "reduced" into single edges.

In [ ]:
plotGraphEnv(oReg.reduce_simple_paths(), env, aImg, figsize=(10,10), node_size=10,
                space=0.3,
                node_colors={"rail":"blue",  "grid":"red"
                             },
                edge_colors={"hold":"gray", "dir":"green"
                             },
                show_nodes=("rail", "grid"),
                show_edges=("dir", "hold"),
                #show_edge_weights=True,
                show_labels=(),
                alpha_img=0.8
               )

In [ ]:
G_reduced = oReg.reduce_simple_paths()
dPos={}
lColor=[]
lFixed=[]
for v, a in G_reduced.nodes(data=True):
    if a.get("type") == "grid":
        lColor.append("red")
        lFixed.append(v)
        dPos[v] = xyPos
    else:
        lColor.append("blue")
    xyPos = v[:2]
    dPos[v] = xyPos


In [ ]:

dPos = nx.spring_layout(G_reduced, pos=dPos, fixed=lFixed, iterations=10000)

nx.draw(G_reduced, with_labels=True, node_size=150, node_color=lColor)

In [ ]:
oReg.G.nodes[(1,10,1)]

In [ ]:
len(env.agents)

In [ ]:
dlPos = {}
for agent in env.agents:
    l = dlPos.get(agent.initial_position, []) + [agent.handle]
    dlPos[agent.initial_position] = l

dlPos

In [ ]:
pilImg = Image.fromarray(aImg)

In [ ]:
with io.BytesIO() as fOut:
    pilImg.save(fOut, format='PNG')
    byteImg = fOut.getvalue()

In [ ]:
lwIm = [ ipw.Image(value=bim) for bim in [byteImg, 
                                          bimgEnv,
                                          bimgReduced]]

In [ ]:
(lGpaths, llnPaths, lltStartTarg) = gu.genStartTargetDirs(G5, env)
#lwImPath = []
for iAgent, Gpath in enumerate(lGpaths):
    plotGraphEnv(Gpath, env, aImg, figsize=(10,10), node_size=10,
                space=0.1,
                node_colors={"rail":"blue", "grid":"red"},
                edge_colors={"hold":"gray", "dir":"green"},
                show_nodes=("rail", "grid"),
                show_edges=("dir"),
                show_labels=(),
                show_edge_weights=True,
                alpha_img=0.7
               )
    with io.BytesIO() as fbOut:
        plt.savefig(fbOut, bbox_inches="tight", pad_inches=0)
        bimgPath = fbOut.getvalue()
    plt.close()
    lwIm.append(ipw.Image(value=bimgPath))
    
    
    

# Pictures of graph, reduced graph, and shortest agent paths

In [ ]:
tab = ipw.Tab(lwIm)
tab.set_title(0, "map")
tab.set_title(1, "graph")
tab.set_title(2, "reduced graph")
for i in range(len(env.agents)):
    tab.set_title(3+i, f"{i}")
tab

### Ibry (+Serjev) Diagram of Resource usage from shortest paths

In [ ]:
dResource, dlRails, dg2Dirs = gu.plotResourceUsage(
    G5, llnPaths, nSteps=100, nStepsShow=25, 
    contradir=False, node_ticks=range(25))
plt.xticks(range(25))
pass

### Hammock of Paths for each agent

In [ ]:
dAgStTg = { iAgent:ltStTg for iAgent,ltStTg in enumerate(lltStartTarg)}

In [ ]:
dlpiHammock = {}
numPaths=0
for iAgent, (nStart, nTarg) in dAgStTg.items():
    lpiHammock = gu.hammockPaths(G5, nStart, nTarg, endSkip=3, preamble=True, ratioMax=2)
    dlpiHammock[iAgent] = lpiHammock
    numPaths += len(lpiHammock)
    print(iAgent, nStart, nTarg, len(lpiHammock), [ oPI.length for oPI in lpiHammock ])
print(numPaths)

In [ ]:
#dlpiHammock[3]

### Pictures of Hammock Paths for each agent

In [ ]:
#### G5c = nx.compose_all([ nx.induced_subgraph(G5, lnPath) for lnPath in lllnHammock[0][:1] + lllnHammock[1][1:2] ])
lwIm = []
for iAgent, lpiHammock in list(dlpiHammock.items())[:10]: # range(len(env.agents)):
    G5c = nx.compose_all([ nx.induced_subgraph(G5, oPI.tPath) for oPI in lpiHammock ])
    gu.plotGraphEnv(G5c, env, aImg, node_size=2, figsize=(10,10), alpha_img=0.7, space=0, show_edge_weights=True)
    with io.BytesIO() as fbOut:
        plt.savefig(fbOut, bbox_inches="tight", pad_inches=0)
        bimgPath = fbOut.getvalue()
    plt.close()
    lwIm.append(ipw.Image(value=bimgPath))

oTab = ipw.Tab(lwIm)

list(map(oTab.set_title, *zip(*[(i, str(i)) for i in range(len(lwIm)) ])))
oTab

### scratch comparing paths / hammocks

In [ ]:
iAgent=0
lpiHammock = dlpiHammock[iAgent]
#G5c = nx.compose_all([ nx.induced_subgraph(G5, oPI.tPath) for oPI in lpiHammock[2:3] ])
#gu.plotGraphEnv(G5c, env, aImg, node_size=2, figsize=(10,10), alpha_img=0.7, space=0, show_edge_weights=True)

#G5c = nx.compose_all([ nx.induced_subgraph(G5, oPI.tPath) for oPI in lpiHammock[0:1] ])
oPI = lpiHammock[0]

oPI2 = dlpiHammock[1][0]

lnGrid = gu.grid_node_for_rails(G5, oPI.tPath)
G5c = nx.induced_subgraph(G5, oPI.tPath + lnGrid + oPI2.tPath) 
gu.plotGraphEnv(G5c, env, aImg, node_size=20, figsize=(15,15), alpha_img=0.7, space=0.2,
    show_edge_weights=True, show_labels=["grid"], show_edges=["dir",], show_nodes=["grid", "rail"], arrowsize=20,
    lvHighlight=[(1,12),(1,13)])
#[ (n,d) for n,d in G5c.nodes(data=True) if d["type"]=="grid" ]

In [ ]:
#list(G5c.edges(data=True))

In [ ]:
iAgent=1
lpiHammock = dlpiHammock[iAgent]
#G5c = nx.compose_all([ nx.induced_subgraph(G5, oPI.tPath) for oPI in lpiHammock[0:1] ])
oPI = lpiHammock[0]
lnGrid = gu.grid_node_for_rails(G5, oPI.tPath)
G5c = nx.induced_subgraph(G5, oPI.tPath + lnGrid) 

gu.plotGraphEnv(G5c, env, aImg, node_size=5, figsize=(10,10), alpha_img=0.7, space=0.4, show_edge_weights=True,
    show_labels=["grid"],
    show_nodes=["grid", "rail", ], 
    show_edges=[
        "dir",
        "hold"
        ])
#print(oPI.tPath)
#list(G5c.edges(data=True))
#[ (n,d) for n,d in G5c.nodes(data=True) if d["type"]=="grid" ]

In [ ]:
G5[ (1,2) ]

In [ ]:
G2.in_edges((1,2))


In [ ]:
G2.out_edges((1,2))

In [ ]:
G2.edges[((1,2),(1,3))]

In [ ]:
intTrans=0b1100000000100010
print(gu.trans_int_to_binstr(intTrans))
print(gu.trans_int_to_4x4(intTrans))
print(gu.trans_int_to_nesw(intTrans))

In [ ]:
import ipycytoscape as ipc

In [ ]:
G_reduced.nodes

In [ ]:
G_reduced.edges(data=True)

In [ ]:
class Node2(ipc.Node):
    def __init__(self, tV, dAttr):
        super().__init__()
        self.data["id"] = str(tV)
        self.data["label"] = str(tV)
        self.classes = dAttr["type"]

class Edge2(ipc.Edge):
    def __init__(self, source, target, data):
        super().__init__()
        self.data["source"] = str(source)
        self.data["target"] = str(target)
        self.classes = data["type"]
        self.data.update(data)


lNodes = [ Node2(v, dAttr) for v, dAttr in G_reduced.nodes(data=True) ]
#lEdges = [ ipc.Edge(source=str(u), target=str(v), data={"source":str(u), "target":str(v)}) for u,v in G_reduced.edges() ]
lEdges = [
    Edge2(source=u, target=v, data=d)
    for u,v,d in G_reduced.edges(data=True) ]
lEdges[0].data

In [ ]:
wC = ipc.CytoscapeWidget()
wC.graph.add_nodes(lNodes)
wC.graph.add_edges(lEdges, directed=True)
wC.graph.nodes

In [ ]:
wC.graph.edges

In [ ]:
wC

In [ ]:
#wC.get_style()
wC.set_style([
    {'selector': 'node', 'css': {'background-color': '#11479e'}},
    {'selector': 'node.rail', 'style': { 'background-color': 'blue' }},
    {'selector': 'node.grid','style': {'background-color': 'red',  'label': 'data(label)'}, },
    {'selector': 'edge', 'style': { 'line-color': 'blue'}},
    {'selector': 'edge.directed',
        'style': {'curve-style': 'bezier',
        'target-arrow-shape': 'triangle',
        'arrow-scale': '2',
        #'target-arrow-color': '#9dbaea'
        }},

    {'selector': 'edge.hold', 'style': { 'line-color': 'skyblue', 'target-arrow-color': 'skyblue', }},
    {'selector': 'edge.grid', 'style': { 'line-color': 'skyblue', 'target-arrow-color': 'skyblue', 'arrow-scale':'0.5' }},
    
    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}},
    #{'selector': 'edge', 'style': {'width': 4, 'line-color': 'red'}},
    {'selector': 'edge.multiple_edges', 'style': {'curve-style': 'bezier'}}
    ])

In [ ]:
wC.relayout()

In [ ]:
#oCW.graph.edges